# <font style="color: rgb(50, 120, 229)"> Implementar una red neuronal en Esp32 </font>

La IA en el borde (Edge AI) es el proceso de ejecutar algoritmos de inteligencia artificial en dispositivos en el borde de Internet u otras redes. El enfoque tradicional para la IA y el aprendizaje automático es usar servidores potentes basados en la nube para realizar el entrenamiento del modelo así como la inferencia (predicciones).

Aunque los dispositivos en el borde pueden tener recursos limitados en comparación con sus homólogos basados en la nube, ofrecen una reducción en el uso del ancho de banda, menor latencia y una mayor privacidad de los datos.


<font style="color: rgb(50, 120, 229)"> Objectivos </font>

En esta práctica vamos a implementar una red neuronal en un ESP32, un microcontrolador de bajo costo y bajo consumo de energía. La red neuronal que vamos a implementar es un clasificador de poses de la mano, que puede clasificar entre 5 poses diferentes (arriba, abajo, izquierda, derecha y centro).

Entrenaremos el modelo utilizando los datos del sensor MPU6050 adquiridos en la práctica anterior.

### <font style="color: rgb(50, 120, 229)"> 1.1 Importar los datos </font>

Vamos a importar los datos que adquirimos en la práctica anterior utilizando la librería pandas.

In [14]:
import pandas as pd

data = pd.read_csv("./data/datos_mpu6050.csv")#TODO: Cargar el dataset usando pandas

#Mezclamos los datos para que no haya sesgo
data = data.sample(frac=1)

data.head()

,acelX,acelY,acelZ,output
688,1.23,9.25,2.98,RIGHT
1601,9.59,-2.26,-1.41,DOWN
492,0.26,8.10,5.38,RIGHT
430,1.87,9.53,1.24,RIGHT
1381,0.35,-9.76,-1.97,LEFT


### <font style="color: rgb(50, 120, 229)"> 1.2 Preprocesar los datos </font>

Vamos a realizar los siguientes pasos para preprocesar los datos:

- Separar los datos en entrada y salida.
- Codificar las etiquetas de salida aplicando one-hot encoding.
- Separar los datos en entrenamiento y prueba.

In [15]:
#TODO: Crear un dataframe con las columnas acelX, acelY y acelZ, guardalo en la variable X


,acelX,acelY,acelZ
688,1.23,9.25,2.98
1601,9.59,-2.26,-1.41
492,0.26,8.10,5.38
430,1.87,9.53,1.24
1381,0.35,-9.76,-1.97


In [16]:
#TODO: Crear un dataframe con la columna output, guardalo en la variable y


688      RIGHT
1601      DOWN
492      RIGHT
430      RIGHT
1381      LEFT
Name: output, dtype: object

Para codificar las etiquetas de salida vamos a utilizar la función `get_dummies` de pandas.

```python
import pandas as pd

encoded_labels = pd.get_dummies(labels, dtype=dtype)
```

- `labels`: es un arreglo de numpy o una serie de pandas con las etiquetas de salida.
- `dtype`: es el tipo de dato de las columnas de la matriz de salida (int, float, etc).

In [17]:
#TODO: Convertir la columna y en un one-hot encoding usando pd.get_dummies, recuerda especificar el tipo de dato como float


,DOWN,IDLE,LEFT,RIGHT,TOP
688,0.0,0.0,0.0,1.0,0.0
1601,1.0,0.0,0.0,0.0,0.0
492,0.0,0.0,0.0,1.0,0.0
430,0.0,0.0,0.0,1.0,0.0
1381,0.0,0.0,1.0,0.0,0.0


Un paso muy importante es separar los datos en entrenamiento y prueba, esto nos permitirá evaluar el modelo en datos que no ha visto durante el entrenamiento.

Realizaremos la separación utilizando la función `sample` de pandas.

```python
train_data = data.sample(frac=0.8)
test_data = data.drop(train_data.index)
```

- **frac**: es el porcentaje de datos que se utilizarán para entrenamiento. En este caso, el 80% de los datos se utilizarán para entrenamiento.

- **drop**: Elimina las filas con los índices especificados, en este caso, eliminamos las filas que se utilizaron para entrenamiento.

In [18]:
#TODO: Dividir los datos en entrenamiento y prueba usando X.sample y y.sample
# Guarda los datos de entrenamiento en las variables X_train y y_train
# Guarda los datos de prueba en las variables X_test y y_test


Por último, vamos a normalizar los datos de entrada utilizando la librería `pandas`.

La normalización que vamos a utilizar se conoce como `estandarización` y se calcula utilizando la siguiente fórmula:

$$
X_{std} = \frac{X - \mu}{\sigma}
$$

Donde:

- $X_{std}$: es el valor normalizado.
- $X$: es el valor original.
- $\mu$: es la media de los datos.
- $\sigma$: es la desviación estándar de los datos.

**La estandarización se aplica a todos los datos, pero el cálculo de la media y la desviación estándar se realiza solo en los datos de entrenamiento.**

In [19]:


print(f"MEAN: {mean.values}")
print(f"STD: {std.values}")

MEAN: [ 1.56263941 -0.25539653  2.91141264]
STD: [6.08261283 5.60008756 4.26860499]


**Los valores obtenidos en la celda anterior se utilizaran para normalizar los datos en la ESP32.**

In [20]:
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

X_train.head()

,acelX,acelY,acelZ
787,-1.795715,-0.002608,-0.363447
1649,1.240809,0.042034,-1.808884
1508,1.087914,0.054534,0.763385
1606,1.385484,0.004535,-1.270067
116,-0.008654,-0.024036,1.627836


**Keras ya proporciona una capa de normalización que se puede utilizar en la red neuronal, no la utilizamos en este caso por que TF Lite Micro aun no soporta esta capa de Keras.**

Hasta este punto, hemos realizado todos los pasos necesarios para preprocesar los datos, pero tenemos los datos en un formato de pandas, necesitamos convertirlos a un formato que pueda ser utilizado por Keras.

Vamos a convertir los datos a un arreglo de numpy utilizando el atributo `values`.

```python
train_data = train_data.values
test_data = test_data.values
```

In [21]:
#TODO: Convertir los datos de entrenamiento y prueba a un arreglo de numpy usando .values

## <font style="color: rgb(50, 120, 229)"> 2. Crear el modelo </font>

Ya que hemos preprocesado los datos, vamos a crear el modelo de la red neuronal.

El modelo que vamos a crear es un modelo secuencial, que consta de las siguientes capas:

- Capa de entrada.
- Capa densa con 8 neuronas y función de activación ReLU.
- Capa de salida con 5 neuronas y función de activación softmax.

```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Input(shape=(3,)))
model.add(Dense(8, activation='relu'))
```

- `Input`: Capa de entrada.
- `Dense`: Capa densa.
- `shape`: Forma de los datos de entrada.
- `activation`: Función de activación.

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

#TODO: Crear un modelo secuencial

#TODO: Agregar una capa de entrada con 3 neuronas, una para cada columna de X

#TODO: Agregar una capa densa con 8 neuronas y activación relu

#TODO: Agregar una capa densa con 5 neuronas y activación softmax, 5 neuronas porque tenemos 5 clases en la salida

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 8)              │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │            45 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77 (308.00 B)

 Trainable params: 77 (308.00 B)

 Non-trainable params: 0 (0.00 B)

Después de definir la arquitectura de la red, vamos a compilar el modelo.

```python
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
```

- `optimizer`: Optimizador.
- `loss`: Función de pérdida, en este caso, utilizamos la entropía cruzada categórica porque estamos realizando una clasificación multiclase.
- `metrics`: Métricas que se utilizarán para evaluar el modelo.

Por último, vamos a entrenar el modelo.

```python
model.fit(train_data, train_labels, epochs=20, validation_data=(test_data, test_labels))
```

- `epochs`: Número de épocas.
- `validation_data`: Datos de validación.


Epoch 1/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5313 - loss: 1.3870 - val_accuracy: 0.6559 - val_loss: 1.2495
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 938us/step - accuracy: 0.6948 - loss: 1.2283 - val_accuracy: 0.7921 - val_loss: 1.0904
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 625us/step - accuracy: 0.7938 - loss: 1.0721 - val_accuracy: 0.8020 - val_loss: 0.9314
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - accuracy: 0.8143 - loss: 0.9032 - val_accuracy: 0.8020 - val_loss: 0.7790
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 938us/step - accuracy: 0.8444 - loss: 0.7254 - val_accuracy: 0.8119 - val_loss: 0.6398
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - accuracy: 0.8638 - loss: 0.5930 - val_accuracy: 0.9431 - val_loss: 0.5177
Epoch 7/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 639us/step - accuracy: 0.9813 - loss: 0.4666 - val_accuracy: 0.9728 - val_loss: 0.4140
Epoch 8/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step - accuracy: 0.9920 - loss: 0.3772 - val_accuracy: 0.99

**Si el modelo se entrena correctamente debes de obtener un valor de perdida bajo y un valor de precisión alto, ádemas de que la precisión en los datos de validación debe ser similar a la precisión en los datos de entrenamiento.**

## <font style="color: rgb(50, 120, 229)"> 3. Convertir el modelo a TensorFlow Lite </font>

Una vez que hemos entrenado el modelo, vamos a convertirlo a TensorFlow Lite para poder ejecutarlo en la ESP32.

Para convertir el modelo a TensorFlow Lite, vamos a utilizar la función `TFLiteConverter` de TensorFlow.

In [25]:
from tensorflow import lite as tflite

model_name = "mpu6050_model" #Nombre del archivo donde se guardará el modelo

converter = tflite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert() #Convertimos el modelo a un modelo tflite

with open(f"{model_name}.tflite", 'wb') as f: #Abrimos un archivo en modo escritura binaria
    f.write(tflite_model) #Guardamos el modelo en un archivo llamado model.tflite

INFO:tensorflow:Assets written to: C:\Users\97ped\AppData\Local\Temp\tmp8ociq3vk\assets


INFO:tensorflow:Assets written to: C:\Users\97ped\AppData\Local\Temp\tmp8ociq3vk\assets


Saved artifact at 'C:\Users\97ped\AppData\Local\Temp\tmp8ociq3vk'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 3), dtype=tf.float32, name='keras_tensor_4')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  1516282288400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1516282287056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1516282287824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1516282289360: TensorSpec(shape=(), dtype=tf.resource, name=None)


Muchas plataformas de microcontroladores no tienen soporte para TensorFlow Lite. La forma más sencilla de ejecutar un modelo de TensorFlow Lite en un microcontrolador es convertirlo a una matriz de bytes y ejecutarlo en el microcontrolador.


In [29]:
def tflite_to_array(model_data, model_name):
    c_str = ""

    #Creamos las cabeceras del archivo
    c_str += f"#ifndef {model_name.upper()}_H\n"
    c_str += f"#define {model_name.upper()}_H\n\n"

    #Agregamos una variable con el tamaño del modelo
    c_str += f"const unsigned int {model_name}_len = {len(model_data)};\n\n"

    #Agregamos el modelo como un arreglo de bytes
    c_str += f"const unsigned char {model_name}[] = {{\n"

    for i, byte in enumerate(model_data):
        c_str += f"0x{byte:02X},"
        if (i + 1) % 12 == 0:
            c_str += "\n"

    c_str += "};\n\n"

    #Cerramos las cabeceras del archivo
    c_str += f"#endif // {model_name.upper()}_H\n"

    return c_str

In [30]:
model_array = tflite_to_array(tflite_model, model_name)

with open(f"{model_name}.h", 'w') as f:
    f.write(model_array)